In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from IPython.display import display, clear_output
import yfinance as yf
from backtesting import Backtest, Strategy
import pandas_ta as ta
import requests
import defs
from datetime import datetime, timedelta



In [62]:
# List of FX pairs to process
fx_jpy_pairs = ['AUD_JPY', 'NZD_JPY', 'EUR_JPY', 'GBP_JPY', 'USD_JPY']

# Dictionary to store DataFrames, key is the FX pair name
jpy_dataframes = {}

# Initialize a session
session = requests.Session()

for instrument in fx_pairs:
    # Set the URL for the request
    url = f'{defs.OANDA_URL}/instruments/{instrument}/candles'

    # Request parameters
    params = {
        'count': 60,  # Number of candles to fetch
        'granularity': 'H1',  # 1-hour candles
        'price': "M",  # M for mid prices
    }

    # Send the request
    response = session.get(url, params=params, headers=defs.SECURE_HEADER)

    # Process the response
    if response.status_code == 200:
        data = response.json()

        # Prepare the data for DataFrame
        my_data = []
        for candle in data['candles']:
            # Skip incomplete candles
            if not candle['complete']:
                continue

            # Extract and store the mid price OHLC and time
            ohlc_data = {
                'time': candle['time'],
                'open': float(candle['mid']['o']),
                'high': float(candle['mid']['h']),
                'low': float(candle['mid']['l']),
                'close': float(candle['mid']['c']),
                'volume': candle['volume']
            }
            my_data.append(ohlc_data)

        # Create DataFrame
        jpy_df = pd.DataFrame(my_data)

        # Convert time to datetime format and set as index
        jpy_df['time'] = pd.to_datetime(jpy_df['time'])
        jpy_df.set_index('time', inplace=True)

        # Adding the SMA and momentum columns
        jpy_df['SMA_FAST'] = ta.sma(jpy_df['close'], length=20).round(3)
        jpy_df['SMA_SLOW'] = ta.sma(jpy_df['close'], length=50).round(3)
        jpy_df['MOMO'] = np.where(jpy_df['SMA_FAST'] > jpy_df['SMA_SLOW'], 'BULL', 'BEAR')

        # Store DataFrame in dictionary
        jpy_dataframes[instrument] = jpy_df
    else:
        print(f"Failed to fetch data for {instrument}: {response.status_code}")

# At this point, fx_dataframes dictionary contains all the DataFrames
# Access a specific DataFrame with fx_dataframes['EUR_USD'], for example


In [65]:
jpy_dataframes['USD_JPY'].tail()

,open,high,low,close,volume,SMA_FAST,SMA_SLOW,MOMO
time,,,,,,,,
2024-03-12 21:00:00+00:00,147.660,147.672,147.626,147.636,682,147.560,147.157,BULL
2024-03-12 22:00:00+00:00,147.638,147.646,147.566,147.583,1893,147.568,147.167,BULL
2024-03-12 23:00:00+00:00,147.584,147.654,147.552,147.622,2222,147.579,147.178,BULL
2024-03-13 00:00:00+00:00,147.622,147.638,147.368,147.374,6413,147.579,147.184,BULL
2024-03-13 01:00:00+00:00,147.372,147.450,147.234,147.423,7057,147.574,147.197,BULL


In [71]:
# List of FX pairs to process
fx_pairs = ['AUD_USD', 'AUD_CAD', 'NZD_USD', 'EUR_USD', 'GBP_USD', 'USD_CAD']

# Dictionary to store DataFrames, key is the FX pair name
fx_dataframes = {}

# Initialize a session
session = requests.Session()

for instrument in fx_pairs:
    # Set the URL for the request
    url = f'{defs.OANDA_URL}/instruments/{instrument}/candles'

    # Request parameters
    params = {
        'count': 60,  # Number of candles to fetch
        'granularity': 'H1',  # 1-hour candles
        'price': "M",  # M for mid prices
    }

    # Send the request
    response = session.get(url, params=params, headers=defs.SECURE_HEADER)

    # Process the response
    if response.status_code == 200:
        data = response.json()

        # Prepare the data for DataFrame
        my_data = []
        for candle in data['candles']:
            # Skip incomplete candles
            if not candle['complete']:
                continue

            # Extract and store the mid price OHLC and time
            ohlc_data = {
                'time': candle['time'],
                'open': float(candle['mid']['o']),
                'high': float(candle['mid']['h']),
                'low': float(candle['mid']['l']),
                'close': float(candle['mid']['c']),
                'volume': candle['volume']
            }
            my_data.append(ohlc_data)

        # Create DataFrame
        fx_df = pd.DataFrame(my_data)

        # Convert time to datetime format and set as index
        fx_df['time'] = pd.to_datetime(fx_df['time'])
        fx_df.set_index('time', inplace=True)

        # Adding the SMA and momentum columns
        fx_df['SMA_FAST'] = ta.sma(fx_df['close'], length=20).round(5)
        fx_df['SMA_SLOW'] = ta.sma(fx_df['close'], length=50).round(5)
        fx_df['MOMO'] = np.where(fx_df['SMA_FAST'] > fx_df['SMA_SLOW'], 'BULL', 'BEAR')

        # Store DataFrame in dictionary
        fx_dataframes[instrument] = fx_df
    else:
        print(f"Failed to fetch data for {instrument}: {response.status_code}")

# At this point, fx_dataframes dictionary contains all the DataFrames
# Access a specific DataFrame with fx_dataframes['EUR_USD'], for example


In [72]:
fx_dataframes['AUD_CAD'].tail()

,open,high,low,close,volume,SMA_FAST,SMA_SLOW,MOMO
time,,,,,,,,
2024-03-12 21:00:00+00:00,0.89125,0.89135,0.89072,0.89135,643,0.89125,0.89156,BEAR
2024-03-12 22:00:00+00:00,0.89136,0.89168,0.89118,0.89138,474,0.89124,0.89152,BEAR
2024-03-12 23:00:00+00:00,0.89138,0.89154,0.89121,0.89154,442,0.89124,0.89148,BEAR
2024-03-13 00:00:00+00:00,0.89158,0.89196,0.89105,0.89112,1011,0.89123,0.89144,BEAR
2024-03-13 01:00:00+00:00,0.89112,0.89140,0.89083,0.89114,1787,0.89124,0.89140,BEAR


In [76]:
last_momo_values = {}

for pair, df in fx_dataframes.items():
    last_momo_value = df['MOMO'].iloc[-1]
    last_momo_values[pair] = last_momo_value


last_momo_jpy_values = {}

for pair, df in jpy_dataframes.items():
    last_momo_jpy_value = df['MOMO'].iloc[-1]
    last_momo_jpy_values[pair] = last_momo_jpy_value



In [78]:
print(last_momo_values)


{'AUD_USD': 'BEAR', 'AUD_CAD': 'BEAR', 'NZD_USD': 'BEAR', 'EUR_USD': 'BEAR', 'GBP_USD': 'BEAR', 'USD_CAD': 'BULL'}


In [95]:
last_momo_jpy_values


{'AUD_JPY': 'BULL',
 'NZD_JPY': 'BULL',
 'EUR_JPY': 'BULL',
 'GBP_JPY': 'BULL',
 'USD_JPY': 'BULL'}